In [1]:
%pip install -Uq tensorflow
%pip install -Uq nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.tokenize import word_tokenize
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
data = pd.read_csv('banks.csv', sep='\t')

In [4]:
class NLTKTokenizer:
    def tokenize_text(self, text):
        return word_tokenize(text)
    
tokenizer_nltk = NLTKTokenizer()
data['text_nltk'] = data['Text'].apply(tokenizer_nltk.tokenize_text)

In [5]:
vectorizer_nltk = CountVectorizer(tokenizer=tokenizer_nltk.tokenize_text)
X_vectorized_nltk = vectorizer_nltk.fit_transform(data['Text'])

c:\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [6]:
X_train_nltk, X_test_nltk, y_train_nltk, y_test_nltk = train_test_split(X_vectorized_nltk, data['Score'], test_size=0.5, random_state=35)

In [7]:
logistic_model_nltk = LogisticRegression()
logistic_model_nltk.fit(X_train_nltk, y_train_nltk)
y_pred_logistic_nltk = logistic_model_nltk.predict(X_test_nltk)
accuracy_logistic_nltk = accuracy_score(y_test_nltk, y_pred_logistic_nltk)
print('Accuracy (Logistic Regression - NLTK):', accuracy_logistic_nltk)
print('\nClassification Report (Logistic Regression - NLTK):\n', classification_report(y_test_nltk, y_pred_logistic_nltk))

c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy (Logistic Regression - NLTK): 0.9422857142857143

Classification Report (Logistic Regression - NLTK):
               precision    recall  f1-score   support

    Negative       0.93      0.95      0.94      3477
    Positive       0.95      0.93      0.94      3523

    accuracy                           0.94      7000
   macro avg       0.94      0.94      0.94      7000
weighted avg       0.94      0.94      0.94      7000



In [8]:
tokenizer_keras = Tokenizer()
tokenizer_keras.fit_on_texts(data['Text'])
X_tokenized_keras = tokenizer_keras.texts_to_sequences(data['Text'])

In [9]:
X_padded_keras = pad_sequences(X_tokenized_keras)
y_encoded_keras = data['Score'].values

In [10]:
X_train_keras, X_test_keras, y_train_keras, y_test_keras = train_test_split(X_padded_keras, y_encoded_keras, test_size=0.5, random_state=35)

In [12]:
model_keras = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(tokenizer_keras.word_index) + 1, 100, input_length=X_padded_keras.shape[1]),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_keras.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [19]:
encoder_keras = LabelEncoder()
y_train_keras_encoded = encoder_keras.fit_transform(y_train_keras)
y_test_keras_encoded = encoder_keras.transform(y_test_keras)

In [21]:
model_keras.fit(X_train_keras, y_train_keras_encoded, epochs=5, batch_size=32, validation_split=0.1, verbose=2)
_, accuracy_keras = model_keras.evaluate(X_test_keras, y_test_keras_encoded)
print('Accuracy (Neural Network - Keras):', accuracy_keras)


Epoch 1/5
197/197 - 195s - loss: 0.4510 - accuracy: 0.8068 - val_loss: 0.2358 - val_accuracy: 0.9229 - 195s/epoch - 989ms/step
Epoch 2/5
197/197 - 155s - loss: 0.1442 - accuracy: 0.9489 - val_loss: 0.2142 - val_accuracy: 0.9214 - 155s/epoch - 784ms/step
Epoch 3/5
197/197 - 153s - loss: 0.0401 - accuracy: 0.9906 - val_loss: 0.2355 - val_accuracy: 0.9214 - 153s/epoch - 775ms/step
Epoch 4/5
197/197 - 153s - loss: 0.0082 - accuracy: 0.9992 - val_loss: 0.2610 - val_accuracy: 0.9214 - 153s/epoch - 776ms/step
Epoch 5/5
197/197 - 152s - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.2764 - val_accuracy: 0.9171 - 152s/epoch - 772ms/step
219/219 [==============================] - 46s 212ms/step - loss: 0.2039 - accuracy: 0.9346
Accuracy (Neural Network - Keras): 0.9345714449882507


In [22]:
_, accuracy = model_keras.evaluate(X_train_keras, y_train_keras_encoded)
print('Accuracy: %.1f' % (accuracy * 100))

219/219 [==============================] - 46s 210ms/step - loss: 0.0290 - accuracy: 0.9917
Accuracy: 99.2
